In [14]:
# Load d3blocks
from d3blocks import D3Blocks

# Initialize
d3 = D3Blocks()

# Load example data
df = d3.import_example('energy')

# Plot
d3.sankey(df, filepath='sankey.html')

[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[datazets] >INFO> Import dataset [energy]
[d3blocks] >INFO> Import dataset [energy]
[datazets] >INFO> Downloading [energy.zip] dataset from github source..
[d3blocks] >INFO> Downloading [energy.zip] dataset from github source..
[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[d3blocks] >INFO> Initializing [Sankey]
[d3blocks] >INFO> Create directory: [C:\Users\lukas\AppData\Local\Temp\d3blocks]
[d3blocks] >INFO> filepath is set to [C:\Users\lukas\AppData\Local\Temp\d3blocks\sankey.html]
[d3blocks] >INFO> Convert to DataFrame.
[d3blocks] >INFO> Node properties are set.
[d3blocks] >INFO> Edge properties are set.
[d3blocks] >INFO> Open browser: C:\Users\lukas\AppData\Local\Temp\d3blocks\sankey.html


In [15]:
df

,source,target,weight
0,Agricultural 'waste',Bio-conversion,124.729
1,Bio-conversion,Liquid,0.597
2,Bio-conversion,Losses,26.862
3,Bio-conversion,Solid,280.322
4,Bio-conversion,Gas,81.144
...,...,...,...
63,Thermal generation,District heating,79.329
64,Tidal,Electricity grid,9.452
65,UK land based bioenergy,Bio-conversion,182.010
66,Wave,Electricity grid,19.013


In [27]:
import pandas as pd

self_df = pd.DataFrame(
    {
        'source': ['A', 'B', 'C', 'D', 'I'],
        'target': ['G', 'H', 'I', 'J', 'J'],
        'weight': [2, 2, 1, 1, 1]
    }
)

d3.sankey(self_df, filepath='sankey.html')

[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[d3blocks] >INFO> Initializing [Sankey]
[d3blocks] >INFO> filepath is set to [C:\Users\lukas\AppData\Local\Temp\d3blocks\sankey.html]
[d3blocks] >INFO> Convert to DataFrame.
[d3blocks] >INFO> Node properties are set.
[d3blocks] >INFO> Edge properties are set.
[d3blocks] >INFO> File already exists and will be overwritten: [C:\Users\lukas\AppData\Local\Temp\d3blocks\sankey.html]
[d3blocks] >INFO> Open browser: C:\Users\lukas\AppData\Local\Temp\d3blocks\sankey.html


In [22]:
import sys
import pypsa
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

from plotting_constants import nice_names, color_dict, stack_to_ax

sys.path.append(str(Path.cwd().parent / 'scripts'))
from _helpers import classify_north_south

In [4]:
path = str(Path.cwd().parent / 'summaries' / 'total_summary_revenues_flex_{}.csv')
rev = pd.read_csv(path.format('national'), index_col=0, header=[0,1,2], parse_dates=True)

In [5]:
day = '2024-03-22'

In [6]:
cost_summary = pd.read_csv(Path.cwd().parent / 'results' / f'{day}' / 'system_cost_summary_flex.csv', 
                            index_col=[0,1], parse_dates=[0])
cost_summary.head()

nodal     zonal  national
snapshot                                                
2024-03-22 wholesale        0.320884  0.320920  0.356214
           congestion_rent -0.225221 -0.217957  0.000000
           cfd_payments     0.253306  0.253306  0.329628
           roc_payments     0.527465  0.518570  0.526107
           offer_cost       0.000000  0.000318  0.222232

In [7]:
mp = pd.read_csv(
    Path.cwd().parent / 'results' / f'{day}' / 'marginal_prices_flex.csv',
    index_col=0,
    parse_dates=[0],
    header=[0,1]
    )


In [8]:
idx = pd.IndexSlice

buses = mp.loc[:,idx['zonal',:]].columns.get_level_values(1)

In [9]:
weights = pd.read_csv(Path.cwd().parent / 'data' / 'preprocessed' / 'load_weights.csv', index_col=0)

weights = weights.loc[buses.astype(int)]
weights /= weights.sum()

In [36]:
load = pd.read_csv(
    Path.cwd().parent / 'results' / f'{day}' / 'gb_total_load_flex.csv',
    index_col=0,
    parse_dates=[0]
).rename(columns={'0': 'load'})

load = pd.DataFrame(
    np.outer(load.values, weights.values),
    index=load.index,
    columns=weights.index.astype(str)
    )

load.head()

Bus,4950,4951,4952,4953,4955,4956,4958,4960,4961,4962,...,8515,8529,8565,8575,8578,8649,8744,8747,8838,8860
snapshot,,,,,,,,,,,,,,,,,,,,,
2024-03-22 00:00:00,117.051558,106.012014,105.282094,103.803471,92.978932,160.162038,78.297572,45.619038,63.498405,161.690244,...,230.882548,233.016865,23.895856,220.209515,159.122259,148.442173,14.095747,151.214500,94.348893,134.073544
2024-03-22 00:30:00,117.360687,106.291988,105.560140,104.077613,93.224486,160.585020,78.504353,45.739516,63.666102,162.117263,...,231.492301,233.632255,23.958964,220.791081,159.542496,148.834203,14.132973,151.613853,94.598065,134.427627
2024-03-22 01:00:00,115.703016,104.790657,104.069147,102.607559,91.907729,158.316824,77.395511,45.093465,62.766845,159.827424,...,228.222568,230.332296,23.620553,217.672499,157.289024,146.731982,13.933351,149.472370,93.261907,132.528893
2024-03-22 01:30:00,115.570678,104.670801,103.950116,102.490200,91.802607,158.135746,77.306989,45.041888,62.695054,159.644618,...,227.961534,230.068850,23.593537,217.423532,157.109122,146.564155,13.917414,149.301408,93.155237,132.377311
2024-03-22 02:00:00,111.175710,100.690337,99.997059,98.592661,88.311501,152.122095,74.367127,43.329017,60.310861,153.573588,...,219.292520,221.319698,22.696312,209.155261,151.134512,140.990553,13.388157,143.623713,89.612691,127.343212


In [37]:
network = pypsa.Network(
    Path.cwd().parent / 'results' / f'{day}' / 'network_flex_s_nodal.nc'
)
buses = network.buses[['x', 'y']]
buses.loc[:, ['region']] = buses.apply(lambda x: classify_north_south(x['x'], x['y']), axis=1)

buses.loc[load.columns]

INFO:pypsa.io:Imported network network_flex_s_nodal.nc has buses, carriers, generators, links, loads, storage_units


,x,y,region
Bus,,,
4950,0.303497,52.288323,south
4951,0.045319,51.928178,south
4952,0.050812,51.641033,south
4953,0.266418,51.623985,south
4955,0.611115,51.868012,south
...,...,...,...
8649,-1.724854,52.571342,south
8744,-4.255829,55.697712,north
8747,-2.013245,51.811162,south


Bus,4950,4951,4952,4953,4955,4956,4958,4960,4961,4962,...,8515,8529,8565,8575,8578,8649,8744,8747,8838,8860
snapshot,,,,,,,,,,,,,,,,,,,,,
2024-03-22 00:00:00,117.051558,106.012014,105.282094,103.803471,92.978932,160.162038,78.297572,45.619038,63.498405,161.690244,...,230.882548,233.016865,23.895856,220.209515,159.122259,148.442173,14.095747,151.214500,94.348893,134.073544
2024-03-22 00:30:00,117.360687,106.291988,105.560140,104.077613,93.224486,160.585020,78.504353,45.739516,63.666102,162.117263,...,231.492301,233.632255,23.958964,220.791081,159.542496,148.834203,14.132973,151.613853,94.598065,134.427627
2024-03-22 01:00:00,115.703016,104.790657,104.069147,102.607559,91.907729,158.316824,77.395511,45.093465,62.766845,159.827424,...,228.222568,230.332296,23.620553,217.672499,157.289024,146.731982,13.933351,149.472370,93.261907,132.528893
2024-03-22 01:30:00,115.570678,104.670801,103.950116,102.490200,91.802607,158.135746,77.306989,45.041888,62.695054,159.644618,...,227.961534,230.068850,23.593537,217.423532,157.109122,146.564155,13.917414,149.301408,93.155237,132.377311
2024-03-22 02:00:00,111.175710,100.690337,99.997059,98.592661,88.311501,152.122095,74.367127,43.329017,60.310861,153.573588,...,219.292520,221.319698,22.696312,209.155261,151.134512,140.990553,13.388157,143.623713,89.612691,127.343212
2024-03-22 02:30:00,111.119423,100.639360,99.946432,98.542746,88.266790,152.045079,74.329477,43.307080,60.280327,153.495836,...,219.181497,221.207648,22.684821,209.049370,151.057996,140.919172,13.381379,143.550999,89.567322,127.278741
2024-03-22 03:00:00,105.880710,95.894728,95.234468,93.896958,84.105461,144.876929,70.825221,41.265372,57.438417,146.259291,...,208.848208,210.778836,21.615348,199.193759,143.936382,134.275553,12.750515,136.783302,85.344680,121.278198
2024-03-22 03:30:00,105.673894,95.707418,95.048447,93.713550,83.941179,144.593943,70.686879,41.184769,57.326223,145.973604,...,208.440267,210.367124,21.573127,198.804676,143.655233,134.013274,12.725610,136.516125,85.177977,121.041307
2024-03-22 04:00:00,105.805479,95.826592,95.166802,93.830242,84.045703,144.773991,70.774898,41.236052,57.397606,146.155370,...,208.699817,210.629073,21.599990,199.052228,143.834112,134.180147,12.741456,136.686115,85.284041,121.192027


In [34]:
rev.loc[day].sum()

north  wind      wholesale            7.742574e+06
                 offers               0.000000e+00
                 bids                 5.606641e+06
                 cfd                  2.926302e+06
                 roc                  7.592116e+06
       disp      wholesale            1.560859e+04
                 offers               0.000000e+00
                 bids                 2.281455e+04
                 cfd                  0.000000e+00
                 roc                  0.000000e+00
       hydro     wholesale            6.326626e+05
                 offers               0.000000e+00
                 bids                 4.599367e+04
                 cfd                  0.000000e+00
                 roc                  1.608938e+05
       storage   wholesale            5.938009e+04
                 offers               1.029000e+03
                 bids                 2.582176e+04
                 cfd                  0.000000e+00
                 roc           